In [1]:
import numpy as np
import torch
import cv2
import torchvision
import torchvision.utils as vutils
from torchvision.transforms import Resize
from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import structural_similarity
# From https://discuss.pytorch.org/t/is-there-anyway-to-do-gaussian-filtering-for-an-image-2d-3d-in-pytorch/12351/3
import torch
import math
import torch.nn as nn

def get_gaussian_kernel(kernel_size=3, sigma=2, channels=3):
    # Create a x, y coordinate grid of shape (kernel_size, kernel_size, 2)
    x_coord = torch.arange(kernel_size)
    x_grid = x_coord.repeat(kernel_size).view(kernel_size, kernel_size)
    y_grid = x_grid.t()
    xy_grid = torch.stack([x_grid, y_grid], dim=-1).float()

    mean = (kernel_size - 1)/2.
    variance = sigma**2.

    # Calculate the 2-dimensional gaussian kernel which is
    # the product of two gaussian distributions for two different
    # variables (in this case called x and y)
    gaussian_kernel = (1./(2.*math.pi*variance)) *\
                      torch.exp(
                          -torch.sum((xy_grid - mean)**2., dim=-1) /\
                          (2*variance)
                      )

    # Make sure sum of values in gaussian kernel equals 1.
    gaussian_kernel = gaussian_kernel / torch.sum(gaussian_kernel)

    # Reshape to 2d depthwise convolutional weight
    gaussian_kernel = gaussian_kernel.view(1, 1, kernel_size, kernel_size)
    gaussian_kernel = gaussian_kernel.repeat(channels, 1, 1, 1)

    gaussian_filter = nn.Conv2d(in_channels=channels, out_channels=channels,kernel_size=kernel_size, groups=channels, bias=False, padding=kernel_size//2)

    gaussian_filter.weight.data = gaussian_kernel
    gaussian_filter.weight.requires_grad = False
    
    return gaussian_filter


def calfeatures(path1, path2, size = 256, blur = False):
    resizer = Resize(size)
    img1 = torchvision.io.read_image(path1).float()
    img2 = torchvision.io.read_image(path2).float()
    img1 = resizer(img1)
    img2 = resizer(img2)
    l2 = torch.sqrt(((img1 - img2) ** 2).mean() * 3)
    if blur:
        blurlayer = get_gaussian_kernel()
        img1 = blurlayer(img1.unsqueeze(0)).squeeze()
        img2 = blurlayer(img2.unsqueeze(0)).squeeze()
    gray1 = torchvision.transforms.functional.rgb_to_grayscale(img1)[0].numpy()
    gray2 = torchvision.transforms.functional.rgb_to_grayscale(img2)[0].numpy()

    ssim = structural_similarity(gray1, gray2, data_range=gray2.max() - gray2.min())
    psnr = peak_signal_noise_ratio(img1.numpy(),img2.numpy(),data_range=255)
    return l2, ssim, psnr

In [4]:
head1 = "ref_imgs/cat/flickr_cat_"
head2 = "tmp/"
list1=["000056.jpg", "000108.jpg", "000136.jpg", "000183.jpg"]
list2=["00000.png", "00001.png", "00002.png", "00003.png"]
# calfeatures("ref_imgs/cat/flickr_cat_000136.jpg", "lowdiv/00002.png")
aa= np.zeros(4)
bb = np.zeros(4)
cc = np.zeros(4)
for i in range(4):
    a, b, c = calfeatures(head1 + list1[i], head2 + list2[i], blur =  True)
    aa[i] = a
    bb[i] = b
    cc[i] = c
print(aa.mean(), bb.mean(), cc.mean())

43.765785217285156 0.5435121729341599 21.1600602670642
